In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

/Users/fklein/neuefische/capstone/Capstone_WindPowerPredicting/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
def plot_windrose(df):
    bins = np.linspace(0,24,13)
    labels = [0,2,4,6,8,10,12,14,16,18,20,22]

    df['WS100BIN'] = pd.cut(df['WS100'], bins=bins, labels = labels)

    df_grouped = df.groupby(['WD100CARD','WS100BIN']).count().reset_index()
    
    wd_card=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]

    wd_zeros = np.zeros(len(wd_card))
    df_all_wd_card = pd.DataFrame([wd_card, wd_zeros, wd_zeros])
    df_all_wd_card = df_all_wd_card.T
    df_all_wd_card.columns = ['WD100CARD', 'WS100BIN', 'FREQUENCY']
    
    data_wind = df_grouped[['WD100CARD', 'WS100BIN', 'ZONEID']]
    data_wind.columns = df_all_wd_card.columns
    #print(data_wind.head(50))

    datax = pd.concat([data_wind, df_all_wd_card], axis = 0)
    print(datax.head())
    datax = datax.sort_values(by='WD100CARD', key=card_sorter)
    ws_ls = np.linspace(0,25,26)
    wind_all_speeds = pd.DataFrame([['N']*len(ws_ls), ws_ls, np.zeros(len(ws_ls))]).T
    wind_all_speeds.columns = ['WD100CARD', 'WS100BIN', 'FREQUENCY']
    wind_all_speeds
    datax = pd.concat([wind_all_speeds, df_all_wd_card, data_wind], axis = 0)

    fig = px.bar_polar(datax, 
                   r="FREQUENCY", 
                   theta="WD100CARD",
                   color="WS100BIN", 
                   #template="plotly_black",
                   color_discrete_sequence= px.colors.sequential.Plasma_r)                      
    #fig.show()
    return fig


In [3]:
def card_sorter(column):
    """Sort function"""
    wd_card=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
    correspondence = {card: order for order, card in enumerate(wd_card)}
    return column.map(correspondence)

In [4]:
### TEST
DATA_PATH = '../data/GEFCom2014Data/Wind/raw_data_incl_features.csv'
data = pd.read_csv(DATA_PATH)
plot_windrose(data)

  WD100CARD WS100BIN FREQUENCY
0         E        0       416
1         E        2      1514
2         E        4      2301
3         E        6      2294
4         E        8       660


In [18]:
DATA_PATH = '../data/GEFCom2014Data/Wind/raw_data_incl_features.csv'

#def update_data(day):
## read data
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP']) #, index_col='TIMESTAMP')
data.interpolate(method='bfill', inplace=True)
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175440 entries, 0 to 175439
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ZONEID      175440 non-null  int64         
 1   TIMESTAMP   175440 non-null  datetime64[ns]
 2   TARGETVAR   175434 non-null  float64       
 3   U10         175440 non-null  float64       
 4   V10         175440 non-null  float64       
 5   U100        175440 non-null  float64       
 6   V100        175440 non-null  float64       
 7   HOUR        175440 non-null  int64         
 8   MONTH       175440 non-null  int64         
 9   WEEKDAY     175440 non-null  int64         
 10  IS_HOLIDAY  175440 non-null  int64         
 11  WS10        175440 non-null  float64       
 12  WS100       175440 non-null  float64       
 13  WD10        175440 non-null  float64       
 14  WD100       175440 non-null  float64       
 15  WD100CARD   175440 non-null  object        
 16  WD

,ZONEID,TIMESTAMP,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,IS_HOLIDAY,WS10,WS100,WD10,WD100,WD100CARD,WD10CARD,U100NORM,V100NORM
0,1,2012-01-01 01:00:00,0.000000,2.124600,-2.681966,2.864280,-3.666076,1,1,6,1,3.421530,4.652334,321.614439,321.999735,NW,NW,0.615665,-0.788008
1,1,2012-01-01 02:00:00,0.054879,2.521695,-1.796960,3.344859,-2.464761,2,1,6,1,3.096451,4.154892,305.473680,306.385781,NW,NW,0.805041,-0.593219
2,1,2012-01-01 03:00:00,0.110234,2.672210,-0.822516,3.508448,-1.214093,3,1,6,1,2.795932,3.712577,287.108562,289.088098,WNW,WNW,0.945017,-0.327022
3,1,2012-01-01 04:00:00,0.165116,2.457504,-0.143642,3.215233,-0.355546,4,1,6,1,2.461699,3.234831,273.345160,276.310236,W,W,0.993941,-0.109912
4,1,2012-01-01 05:00:00,0.156940,2.245898,0.389576,2.957678,0.332701,5,1,6,1,2.279435,2.976332,260.159324,263.581938,W,W,0.993733,0.111782


In [41]:
day = '2013-07-01'

def get_data(day):
    day +=' 1:00:00'
    data['TIMESTAMP']
    forecast_start = pd.to_datetime(day)
    forecast_end = forecast_start + pd.Timedelta(value=23, unit='hour')
    retro_start = forecast_start - pd.Timedelta(value=24, unit='hour')
    retro_end = retro_start + pd.Timedelta(value=23, unit='hour')
    data_forecast = data[(data['TIMESTAMP']>=forecast_start) & (data['TIMESTAMP']<=forecast_end)]
    data_retro = data[(data['TIMESTAMP']>=retro_start) & (data['TIMESTAMP']<=retro_end)]
    return data_forecast, data_retro

data_forecast, data_retro = get_data(day)

In [49]:
data_retro.groupby('ZONEID').sum()/24

,TARGETVAR,U10,V10,U100,V100,HOUR,MONTH,WEEKDAY,IS_HOLIDAY,WS10,WS100,WD10,WD100,U100NORM,V100NORM
ZONEID,,,,,,,,,,,,,,,
1,0.593534,3.707042,-1.819963,6.653751,-2.893349,11.5,6.041667,5.75,0.0,4.181666,7.309076,296.765392,293.377544,0.911002,-0.394075
2,0.375511,3.998596,0.472942,7.199071,1.088709,11.5,6.041667,5.75,0.0,4.115849,7.400286,264.019777,260.783503,0.971207,0.157499
3,0.150179,3.464878,-1.332586,6.218612,-2.131158,11.5,6.041667,5.75,0.0,3.768464,6.635192,291.358034,288.385272,0.939285,-0.312943
4,0.265700,3.787608,1.614314,6.165237,2.862546,11.5,6.041667,5.75,0.0,4.147104,6.854207,247.331271,245.297255,0.900427,0.414163
5,0.293427,3.787608,1.614314,6.165237,2.862546,11.5,6.041667,5.75,0.0,4.147104,6.854207,247.331271,245.297255,0.900427,0.414163
6,0.374759,3.716034,0.328386,6.180457,1.033762,11.5,6.041667,5.75,0.0,3.848963,6.420608,267.684019,261.261136,0.964099,0.148730
7,0.690229,5.558717,-0.982004,9.460361,-1.453928,11.5,6.041667,5.75,0.0,5.653308,9.582102,280.282302,278.818706,0.987128,-0.153152
8,0.731638,5.558717,-0.982004,9.460361,-1.453928,11.5,6.041667,5.75,0.0,5.653308,9.582102,280.282302,278.818706,0.987128,-0.153152
9,0.373918,3.964297,-1.847696,7.144898,-3.084388,11.5,6.041667,5.75,0.0,4.436010,7.851026,295.813966,293.242126,0.910696,-0.391425
